## 2.2 - Chain of Thought Prompting

**Chain of Thought** (CoT) is a concept in natural language processing (NLP) that enables 
language models to reason through tasks in a structured, step-by-step manner, similar to human 
thinking. Rather than generating isolated responses, a model using Chain of Thought builds on 
prior information, creating responses that follow a logical flow and ensuring a coherent, 
context-aware dialogue. This approach is particularly valuable in complex or multi-step queries,
where understanding and maintaining context over multiple interactions is essential.

![LLM Tools](../../content/modules/ROOT/assets/images/04/04-02-cot.png)

## Why Chain of Thought Matters

In traditional LLM models, responses can sometimes feel disjointed or context-free, especially with complex questions. Chain of Thought addresses these challenges by adding a reasoning layer, allowing the model to generate more accurate and relevant responses. This logical progression offers key benefits:

1. **Coherence**: Responses follow a natural flow, making it easier for users to follow and engage with the conversation.
2. **Context Retention**: By building on previous information, the model maintains context across multiple steps, crucial for handling complex questions.
3. **Increased Accuracy**: Logical progression reduces the likelihood of irrelevant or incorrect information, enhancing the reliability of responses.


## Preparation

In [1]:
#!pip install -q langchain==0.1.9 openai==1.13.3
# !pip install -q langchain-openai langchain
#!pip uninstall langchain openai

!pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
#from langchain_community.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
# INFERENCE_SERVER_URL = os.getenv('API_URL')
# MODEL_NAME = "mistral-7b-instruct"
# API_KEY= os.getenv('API_KEY')

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.0-8b-instruct"
API_KEY= "alanliuxiang"

### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Create the Prompt

In [5]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct but in a simple way and very concise
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_LLM = PromptTemplate(input_variables=["input"], template=template)

In [6]:
conversation = LLMChain(llm=llm,
                            prompt=PROMPT_LLM,
                            verbose=False,
                            )

/tmp/ipykernel_2493/2670321090.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(llm=llm,


### Let's ask a simple question **WITHOUT** Chain of Thought in action

In [7]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation.predict(input=first_input);

When designing a financial report based on the reports of larger Financial Service Institutions (FSIs), you should focus on the following key areas:

1. **Financial Statements**: Review the income statement, balance sheet, and cash flow statement to understand the institution's financial performance, assets, liabilities, and cash flow.

2. **Risk Management**: Analyze the institution's risk management strategies, including credit risk, market risk, operational risk, and liquidity risk.

3. **Regulatory Compliance**: Ensure the institution adheres to relevant financial regulations and standards, such as Basel III, IFRS, or GAAP.

4. **Profitability and Efficiency**: Evaluate the institution's profitability ratios (e.g., return on assets, return on equity) and efficiency ratios (e.g., cost-to-income ratio, asset turnover).

5. **Capital Adequacy**: Assess the institution's capital adequacy ratio to ensure it has sufficient capital to cover potential losses and meet regulatory requirement

### LLM with Chain of Thought in the Prompt

In [8]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
When answering, think step-by-step and explain your reasoning before providing the final answer.
Explain it with bullet points if it's needed but using reasoning and step-by-step.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_COT = PromptTemplate(input_variables=["input"], template=template)

### Chain of Thought Prompt

Let's ask the same question with Chain of Thought

* When answering, think step-by-step and explain your reasoning before providing the final answer.
* Explain it with bullet points if it's needed but using reasoning and step-by-step.

In [9]:
conversation_cot = LLMChain(llm=llm,
                            prompt=PROMPT_COT,
                            verbose=False,
                            )

In [10]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation_cot.predict(input=first_input);

1. **Understand the Purpose**: The primary purpose of a financial report is to provide information about a company's financial performance and position. It should be clear, concise, and easy to understand.

2. **Identify the Relevant Information**: Look for key financial metrics such as revenue, expenses, profits, losses, assets, liabilities, and equity. Also, consider non-financial information like market trends, competitive landscape, and strategic initiatives.

3. **Compare with Larger FSIs**: Compare your company's financial performance with that of larger Financial Service Institutions (FSIs). This will help you understand your company's position in the market and identify areas for improvement.

4. **Use Standard Formats**: Use standard financial reporting formats to ensure consistency and comparability. This could include income statements, balance sheets, cash flow statements, and statements of shareholders' equity.

5. **Provide Context**: Provide context for your financial pe

## Enhancing LLMs Performance with Chain of Thought

By simulating a thought process, Chain of Thought enhances NLP models by:

- **Improving Context Understanding**: The model remembers previous interactions, generating responses that are relevant and contextually aware.
- **Boosting User Experience**: Users benefit from a more natural and satisfying interaction, as the model “thinks” through its responses.
- **Increasing Flexibility**: CoT enables models to handle multi-part questions, making them versatile for applications like virtual assistants, customer service bots, and educational tools.

Incorporating Chain of Thought into LLMs frameworks, such as LangChain, enables language models to tackle complex tasks, making them more effective for real-world applications where logical reasoning and clarity are paramount.